In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [3]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 30
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [6]:
# import impl.RNN as nn
# net = nn.RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.LSTM as nn
# net = nn.LSTM(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU as nn
# net = nn.GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU2 as nn
# net = nn.GRU2(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

import impl.GRU3 as nn
net = nn.GRU3(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# # GRU4 goes to inf because the res-conn is gone.
# import impl.GRU4 as nn
# net = nn.GRU4(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [7]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step, # This is the width of the network
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3845
hyRmh: hc–bie;o whe 1me Ean ouncMstfitaneSha iael y dtAeaf euntiatm iht N5ecr5iryd sJ Jaundorysina t


Iter-2000 loss: 2.7135
tan y lilllcifin . Npeleer wounssured in o44 ladorgoJty thin on tadys. farl t0ir1'as pipale tor the 


Iter-3000 loss: 2.2756
wifd orrly. 1M6 7h couth iOdkis of the barith. Afrhe Kurofrloyb. Indh inlgcunderec, thicwer tod NeEa


Iter-4000 loss: 1.9765
s an lacacenstimoule ion the Gommaled warh the 12th and SonPiala Oinniy usthercarted fure w1kt Cl on


Iter-5000 loss: 1.7458
 ih 3is laran cofusorke wud in kakun, surthe ,of Japan'ss an oarcan, wintir inss. Inxt. (enaperea, w


Iter-6000 loss: 1.5551
07t ronoky thin minturens of and defitor ofimionl Wonto Son woitl enan micaneh in the erd efthe E"pi


Iter-7000 loss: 1.3895
d War CD anmzlecorldWds verhigd tury opitns uf of 1966, hy Intoblidian teatinded fiontitelead ciract


Iter-8000 loss: 1.2698
 pombliatd of the and pincary wor. as China mboumitol enand ompirtored ansg Runsia tansed in th